In [1]:
import numpy as np
import pandas as pd

In [2]:
input_fname = 'Train_Tagged_Titles.tsv'

In [3]:
df = pd.read_csv(input_fname, sep='\t')

In [4]:
df.drop(['Unnamed: 4'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
# df.head(30)
print(df.iloc[0])

Record Number                                                    1
Title            Supreme Nike SB Dunk High By any Means Red US1...
Token                                                      Supreme
Tag                                                         Modell
Name: 0, dtype: object


In [6]:
print(len(df))

52005


In [7]:
from spacy_langdetect import LanguageDetector
import spacy

In [8]:
df.head()

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52005 entries, 0 to 52004
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Record Number  52005 non-null  object
 1   Title          52004 non-null  object
 2   Token          52001 non-null  object
 3   Tag            42164 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


In [10]:
from spacy.language import Language

In [12]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)
text = 'Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force'
text2 = 'NEU SKECHERS Damen Sneakers Turnschuhe Memory Foam MODA - LAZY SUNDAYS Blau'
doc = nlp(text2)
print(doc._.language)

/opt/anaconda3/envs/ebaynlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'language': 'de', 'score': 0.8571399155765819}


In [13]:
# lim = len(df)
# i = 0

# while i < lim:    
#     row = df.iloc[i]
    
#     if row['Tag'] is np.NaN:
#         df.iloc[i-1]['Token'] = str(df.iloc[i-1]['Token']) + ' ' + str(row['Token'])
#         df.drop([i], inplace=True)
#         df.reset_index(drop=True, inplace=True)
        
#         lim = len(df)
#     else:
#         i += 1